# First Eagle Case - James Li
## 01/09/2026
## Part I
## Data Preparation

In [214]:
#Read Data
import pandas as pd, numpy as np, re, math, os, datetime
file_path = "Input.xlsx"
sheet_name = "Close Price Data"

prices = pd.read_excel(
    file_path,
    sheet_name=sheet_name
)

prices.head()

,Date,FECRX,SGIIX,FEBIX,FEREX,FEGIX,FEHIX,SGOIX,FERLX,FEAIX,FDUIX,FESCX,FESMX,FEVIX,HYG,LQD,SGOV
0,2025-12-23,22.36,81.52,15.79,12.71,61.76,8.03,31.40,25.42,30.77,9.89,11.85,12.98,24.09,80.49,110.22,100.29
1,2025-12-22,22.36,81.21,15.71,12.66,61.35,8.03,31.20,25.42,30.73,9.89,11.88,13.02,24.05,80.43,110.11,100.28
2,2025-12-19,22.36,80.55,15.64,12.54,59.57,8.02,30.99,25.40,30.55,9.88,11.80,12.89,23.85,80.36,110.13,100.28
3,2025-12-18,22.36,80.25,15.64,12.47,58.49,8.03,30.88,25.39,30.46,9.90,11.77,12.77,23.74,80.40,110.41,100.24
4,2025-12-17,22.36,80.00,15.62,12.39,58.40,8.03,30.76,25.39,30.28,9.89,11.70,12.68,23.72,80.18,110.00,100.24


In [215]:
#Calculate Log Return based on close price data
prices["Date"] = pd.to_datetime(prices["Date"])
prices = prices.set_index("Date")
prices = prices.sort_index()
prices = prices.apply(pd.to_numeric, errors="coerce")
log_returns = np.log(prices / prices.shift(1))
log_returns = log_returns.dropna(how="all")

log_returns.head()

,FECRX,SGIIX,FEBIX,FEREX,FEGIX,FEHIX,SGOIX,FERLX,FEAIX,FDUIX,FESCX,FESMX,FEVIX,HYG,LQD,SGOV
Date,,,,,,,,,,,,,,,,
2025-04-03,-0.003303,-0.020354,-0.004508,-0.021730,-0.000880,0.007255,-0.007023,0.000400,-0.027093,0.003011,-0.078902,-0.070869,-0.028658,-0.012530,-0.000095,0.000205
2025-04-04,-0.003788,-0.050956,-0.036814,-0.061043,-0.081291,0.003608,-0.048852,0.000000,-0.049597,0.001002,-0.043105,-0.045275,-0.050629,-0.014321,-0.001045,0.000308
2025-04-07,-0.002850,-0.015878,-0.018132,-0.021550,0.002226,-0.035434,-0.026468,0.000799,-0.007818,-0.012085,-0.008368,-0.008457,-0.007530,-0.008610,-0.019687,0.000103
2025-04-08,0.000951,-0.009010,-0.004785,-0.017792,-0.004457,-0.020101,-0.004031,0.000399,-0.012544,-0.011207,-0.029235,-0.022545,-0.011696,-0.004677,-0.012786,0.000000
2025-04-09,0.001899,0.054226,0.029923,0.055456,0.063062,-0.020514,0.041755,0.000000,0.067777,-0.013409,0.086393,0.083295,0.058824,0.026534,0.016658,0.000205


In [216]:
# Load Annualized Return Data
input_df = pd.read_excel(
    "Input.xlsx",
    sheet_name="Input",
    usecols="B,K",   
    skiprows=4     
)

input_df.columns = ["Ticker", "Return_Annualized"]
mu_annual = (
    input_df
    .set_index("Ticker")["Return_Annualized"]
    .astype(float)
    / 100
)

mu_annual

Ticker
FECRX    0.077297
SGIIX    0.325313
FEBIX    0.222384
FEREX    0.290091
FEGIX    1.101632
FEHIX   -0.014416
SGOIX    0.346261
FERLX    0.087900
FEAIX    0.303718
FDUIX   -0.001142
FESCX    0.382314
FESMX    0.341101
FEVIX    0.262353
HYG      0.102141
LQD      0.091206
SGOV     0.042598
Name: Return_Annualized, dtype: float64

In [217]:
# Calculate Annualzied Cov. Matrix
trading_days = 252 #Assuming 252 trading days in a year
cov_annual = returns.cov() * trading_days
cov_annual.head()


,FECRX,SGIIX,FEBIX,FEREX,FEGIX,FEHIX,SGOIX,FERLX,FEAIX,FDUIX,FESCX,FESMX,FEVIX,HYG,LQD,SGOV
FECRX,0.000543,0.000685,0.000499,0.000785,0.000688,0.000391,0.000530,-0.000379,0.000812,0.000260,0.000921,0.001082,0.000741,0.000302,0.000085,0.000004
SGIIX,0.000685,0.022962,0.012272,0.018426,0.022871,-0.000602,0.018204,-0.000020,0.025647,-0.000525,0.025387,0.023789,0.024215,0.006138,0.004014,-0.000017
FEBIX,0.000499,0.012272,0.008477,0.012247,0.018584,0.000708,0.011454,-0.000073,0.011812,0.000206,0.012974,0.011958,0.011853,0.003331,0.002747,-0.000009
FEREX,0.000785,0.018426,0.012247,0.023769,0.027405,-0.000364,0.017105,-0.000001,0.018552,-0.000451,0.029784,0.026040,0.018071,0.006890,0.005060,-0.000004
FEGIX,0.000688,0.022871,0.018584,0.027405,0.108042,0.000940,0.025587,-0.000094,0.012088,0.000128,0.014408,0.010362,0.018929,0.003210,0.002630,-0.000030


In [218]:
# Formatting
mu_annual.index = mu_annual.index.astype(str).str.strip().str.upper()
w_current.index = w_current.index.astype(str).str.strip().str.upper()

# Manually calculate portfolio expected return 
exp_ret_manual = (w_current * mu_annual.reindex(w_current.index)).sum()
exp_ret_manual


np.float64(0.21563102790000005)

In [219]:
# Calculate current portfolio metrics
def portfolio_stats(weights: pd.Series, mu: pd.Series, cov: pd.DataFrame, rf: float = 0.0) -> dict:
    weights = weights.copy()
    weights.index = weights.index.astype(str).str.strip().str.upper()
    mu = mu.copy()
    mu.index = mu.index.astype(str).str.strip().str.upper()

    cov = cov.copy()
    cov.index = cov.index.astype(str).str.strip().str.upper()
    cov.columns = cov.columns.astype(str).str.strip().str.upper()

    tickers = cov.columns

    w = weights.reindex(tickers).fillna(0.0).values
    mu_v = mu.reindex(tickers).values
    cov_m = cov.values

    port_ret = float(w @ mu_v)
    port_var = float(w @ cov_m @ w)
    port_vol = float(np.sqrt(max(port_var, 0.0)))
    sharpe = (port_ret - rf) / port_vol if port_vol > 0 else np.nan

    return {"return": port_ret, "vol": port_vol, "variance": port_var, "sharpe": sharpe}

current_stats = portfolio_stats(w_current, mu_annual, cov_annual, rf=0.0)
current_stats


{'return': 0.21563102790000002,
 'vol': 0.10819563733051128,
 'variance': 0.011706295937355527,
 'sharpe': 1.9929734065089872}

In [220]:
def risk_contribution(weights: pd.Series, cov: pd.DataFrame) -> pd.DataFrame:
    """
    weights: pd.Series (sum = 1)
    cov: annualized covariance matrix
    """
    weights = weights.reindex(cov.columns).fillna(0.0)

    w = weights.values
    cov_m = cov.values

    # Portfolio volatility
    port_var = float(w @ cov_m @ w)
    port_vol = np.sqrt(max(port_var, 0))

    # Σw
    sigma_w = cov_m @ w

    # MRC, RC, RC Share
    mrc = sigma_w / port_vol
    rc = w * mrc
    rc_share = rc / rc.sum()

    return pd.DataFrame({
        "Weight": weights,
        "Sigma_w": sigma_w,
        "MRC": mrc,
        "Risk_Contribution": rc,
        "Risk_Contribution_Share": rc_share
    })


In [221]:
# Current portfolio risk contribution
rc_current = risk_contribution(w_current, cov_annual)
rc_current

,Weight,Sigma_w,MRC,Risk_Contribution,Risk_Contribution_Share
FECRX,0.00,0.000492,0.004552,0.000000,0.000000
SGIIX,0.09,0.015307,0.141474,0.012733,0.117682
FEBIX,0.00,0.008140,0.075237,0.000000,0.000000
FEREX,0.00,0.013974,0.129155,0.000000,0.000000
FEGIX,0.00,0.012435,0.114935,0.000000,0.000000
FEHIX,0.00,-0.000023,-0.000213,-0.000000,-0.000000
SGOIX,0.08,0.011710,0.108231,0.008658,0.080026
FERLX,0.00,-0.000008,-0.000078,-0.000000,-0.000000
FEAIX,0.00,0.017977,0.166155,0.000000,0.000000
FDUIX,0.07,-0.000131,-0.001206,-0.000084,-0.000781


In [222]:
# Calculate current portfolio risk profile
rc_summary = {
    "Max Risk Contribution Share": rc_current["Risk_Contribution_Share"].max(),
    "Top 3 Risk Share Sum": rc_current["Risk_Contribution_Share"]
        .sort_values(ascending=False)
        .head(3)
        .sum(),
    "Effective Number of Bets (1/HHI)": 1 / (rc_current["Risk_Contribution_Share"]**2).sum()
}

rc_summary

{'Max Risk Contribution Share': 0.4114211356560102,
 'Top 3 Risk Share Sum': np.float64(0.8045397633132061),
 'Effective Number of Bets (1/HHI)': np.float64(3.5855439731230385)}

## Current Portfolio Summary Table

In [223]:
# Current Portfolio Stats Summary Table
current_stats = portfolio_stats(
    w_current,
    mu_annual,
    cov_annual,
    rf=0.0
)

summary_current = pd.DataFrame({
    "Metric": [
        "Portfolio Expected Return (Annualized)",
        "Portfolio Volatility (Annualized)",
        "Portfolio Variance",
        "Sharpe Ratio (Rf = 0)",
        "Max Risk Contribution Share",
        "Top 3 Risk Contribution Share",
        "Effective Number of Bets"
    ],
    "Value": [
        current_stats["return"],
        current_stats["vol"],
        current_stats["variance"],
        current_stats["sharpe"],
        rc_summary["Max Risk Contribution Share"],
        rc_summary["Top 3 Risk Share Sum"],
        rc_summary["Effective Number of Bets (1/HHI)"]
    ]
})

summary_current

,Metric,Value
0,Portfolio Expected Return (Annualized),0.215631
1,Portfolio Volatility (Annualized),0.108196
2,Portfolio Variance,0.011706
3,Sharpe Ratio (Rf = 0),1.992973
4,Max Risk Contribution Share,0.411421
5,Top 3 Risk Contribution Share,0.804540
6,Effective Number of Bets,3.585544


## Optimized Portfolio (Incremental MVO Approach)

In [253]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

def optimize_incremental_portfolio(
    mu_annual: pd.Series,
    cov_annual: pd.DataFrame,
    w_current: pd.Series,
    risk_aversion: float = 3.0,
    max_active_weight: float = 0.10,   # |w_i - w_i,current| cap (per-asset tracking constraint)
    max_asset_weight: float = 0.40,    # long-only + single-name cap
    sgov_min: float = 0.02,
    sgov_max: float = 0.10,
    fehix_max: float = 0.20,
    hyg_max: float = 0.20,
    fegix_max: float = 0.20,
    illiquid_max: float = 0.20,        # portfolio-level cap on illiquid bucket weight
    equity_max: float = 0.60,          # portfolio-level cap on equity bucket weight
    illiquid_tickers: tuple = ("FECRX", "FERLX"),
    equity_tickers: tuple = ("FEGIX", "SGOIX", "FEAIX", "FESCX", "FESMX"),
    verbose: bool = True,
) -> dict:
    """
    Incremental optimizer (tracking-style): maximize mean-variance utility subject to
    per-asset active weight bounds and additional policy constraints.

    Objective (maximize):
        U(w) = mu' w  -  risk_aversion * (w' Sigma w)

    Constraints:
        1) sum(w) = 1
        2) 0 <= w_i <= max_asset_weight
        3) |w_i - w_i,current| <= max_active_weight  for all i
        4) SGOV bounds, plus sector/bucket caps (FEHIX/HYG/FEGIX, illiquid bucket, equity bucket)

    Returns:
        dict with optimal weights (pd.Series) + objective + raw solver output.
    """

    # -----------------------------
    # Align universe ordering (critical for matrix algebra correctness)
    # -----------------------------
    tickers = list(cov_annual.columns)
    cov = cov_annual.loc[tickers, tickers].astype(float)

    mu = mu_annual.reindex(tickers).astype(float)
    if mu.isna().any():
        missing = mu[mu.isna()].index.tolist()
        raise ValueError(f"Missing expected returns for tickers: {missing}")

    w0 = w_current.reindex(tickers).fillna(0.0).astype(float)
    if (w0 < -1e-12).any():
        raise ValueError("Current weights contain negative values; expected long-only.")
    if w0.sum() <= 0:
        raise ValueError("Current weights sum to zero; cannot initialize optimizer.")
    w0 = w0 / w0.sum()

    mu_v = mu.values
    Sigma = cov.values
    w0_v = w0.values
    n = len(tickers)

    # -----------------------------
    # Objective: maximize utility => minimize negative utility
    # -----------------------------
    def neg_utility(w: np.ndarray) -> float:
        w = np.asarray(w, dtype=float)
        port_ret = float(w @ mu_v)
        port_var = float(w @ Sigma @ w)
        utility = port_ret - 0.5*risk_aversion * port_var
        return -utility

    # -----------------------------
    # Constraints
    # -----------------------------
    constraints = []

    # Fully invested constraint: sum(w) = 1
    constraints.append({"type": "eq", "fun": lambda w: np.sum(w) - 1.0})

    # Per-asset active weight constraint: |w_i - w0_i| <= max_active_weight
    # Implemented as two linear inequalities:
    #   w_i - w0_i <= max_active_weight
    #  -(w_i - w0_i) <= max_active_weight
    for i in range(n):
        constraints.append({"type": "ineq", "fun": lambda w, i=i: max_active_weight - (w[i] - w0_v[i])})
        constraints.append({"type": "ineq", "fun": lambda w, i=i: max_active_weight + (w[i] - w0_v[i])})

    # SGOV allocation band (if present in the universe)
    if "SGOV" in tickers:
        idx = tickers.index("SGOV")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: w[idx] - sgov_min})
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: sgov_max - w[idx]})

    # Single-fund caps (policy constraints)
    if "FEHIX" in tickers:
        idx = tickers.index("FEHIX")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: fehix_max - w[idx]})

    if "HYG" in tickers:
        idx = tickers.index("HYG")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: hyg_max - w[idx]})

    if "FEGIX" in tickers:
        idx = tickers.index("FEGIX")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: fegix_max - w[idx]})

    # Bucket caps: illiquid and equity exposure
    illiquid_idx = [tickers.index(t) for t in illiquid_tickers if t in tickers]
    if len(illiquid_idx) > 0:
        constraints.append({"type": "ineq", "fun": lambda w, ii=illiquid_idx: illiquid_max - np.sum(w[ii])})

    equity_idx = [tickers.index(t) for t in equity_tickers if t in tickers]
    if len(equity_idx) > 0:
        constraints.append({"type": "ineq", "fun": lambda w, ei=equity_idx: equity_max - np.sum(w[ei])})

    # -----------------------------
    # Bounds: long-only and single-name cap
    # -----------------------------
    bounds = [(0.0, max_asset_weight) for _ in range(n)]

    # -----------------------------
    # Initialization: start from current weights (projected into bounds)
    # -----------------------------
    x0 = np.clip(w0_v.copy(), 0.0, max_asset_weight)
    x0 = x0 / x0.sum()

    # -----------------------------
    # Solve
    # -----------------------------
    res = minimize(
        fun=neg_utility,
        x0=x0,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": 3000, "ftol": 1e-12, "disp": verbose},
    )

    if not res.success:
        raise RuntimeError(f"Incremental optimization failed: {res.message}")

    w_opt = pd.Series(res.x, index=tickers, name="Weight")

    return {
        "weights": w_opt,
        "objective_value": float(-res.fun),  # utility = mu'w - 0.5*risk_aversion * w'Σw
        "solver_result": res,
    }

In [254]:
def format_portfolio_summary(stats: dict) -> dict:
    """Format key portfolio statistics for a finance-style summary table."""
    return {
        "Expected Return (Annualized)": f"{stats['return']:.2%}",
        "Volatility (Annualized)": f"{stats['vol']:.2%}",
        "Variance (Annualized)": f"{stats['variance']:.4f}",
        "Sharpe (Rf=0)": f"{stats['sharpe']:.2f}",
    }

# 1) Solve incremental portfolio
inc = optimize_incremental_portfolio(
    mu_annual=mu_annual,
    cov_annual=cov_annual,
    w_current=w_current,
    risk_aversion=3.0,
    max_active_weight=0.10,
    max_asset_weight=0.40,
    illiquid_tickers=("FECRX", "FERLX"),  # update if your Private Debt set differs
    equity_tickers=("FEGIX", "SGOIX", "FEAIX", "FESCX", "FESMX"),  # update if your Equity set differs
    verbose=True,
)

w_inc = inc["weights"]

# 2) Compute stats (assumes your portfolio_stats() is already correct)
stats_cur = portfolio_stats(w_current, mu_annual, cov_annual, rf=0.0)
stats_inc = portfolio_stats(w_inc, mu_annual, cov_annual, rf=0.0)

# 3) Finance-style summary table
summary = pd.DataFrame({
    "Metric": list(format_portfolio_summary(stats_cur).keys()),
    "Current": list(format_portfolio_summary(stats_cur).values()),
    "Incremental": list(format_portfolio_summary(stats_inc).values()),
})

summary

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.326967397960692
            Iterations: 5
            Function evaluations: 85
            Gradient evaluations: 5


,Metric,Current,Incremental
0,Expected Return (Annualized),21.56%,35.36%
1,Volatility (Annualized),10.82%,13.33%
2,Variance (Annualized),0.0117,0.0178
3,Sharpe (Rf=0),1.99,2.65


## Check if constraints are met

In [255]:
def incremental_constraint_report(
    w_opt: pd.Series,
    w_current: pd.Series,
    max_asset_weight: float = 0.40,
    max_active_weight: float = 0.10,
    sgov_min: float = 0.02,
    sgov_max: float = 0.10,
    fehix_max: float = 0.20,
    hyg_max: float = 0.20,
    fegix_max: float = 0.20,
    illiquid_tickers: tuple = ("FECRX", "FERLX"),
    equity_tickers: tuple = ("FEGIX", "SGOIX", "FEAIX", "FESCX", "FESMX"),
    illiquid_max: float = 0.20,
    equity_max: float = 0.60,
) -> pd.Series:
    """Constraint sanity checks to confirm implementation integrity."""
    w = w_opt.copy()
    w0 = w_current.reindex(w.index).fillna(0.0)

    out = {
        "Sum of weights": float(w.sum()),
        "Max single-name weight": float(w.max()),
        "Min single-name weight": float(w.min()),
        "Max |active weight|": float((w - w0).abs().max()),
    }

    if "SGOV" in w.index:
        out["SGOV weight"] = float(w["SGOV"])
        out["SGOV within band"] = (w["SGOV"] >= sgov_min - 1e-8) and (w["SGOV"] <= sgov_max + 1e-8)

    if "FEHIX" in w.index:
        out["FEHIX cap OK"] = float(w["FEHIX"]) <= fehix_max + 1e-8
    if "HYG" in w.index:
        out["HYG cap OK"] = float(w["HYG"]) <= hyg_max + 1e-8
    if "FEGIX" in w.index:
        out["FEGIX cap OK"] = float(w["FEGIX"]) <= fegix_max + 1e-8

    ill_w = float(w[[t for t in illiquid_tickers if t in w.index]].sum())
    eq_w  = float(w[[t for t in equity_tickers if t in w.index]].sum())
    out["Illiquid bucket weight"] = ill_w
    out["Illiquid cap OK"] = ill_w <= illiquid_max + 1e-8
    out["Equity bucket weight"] = eq_w
    out["Equity cap OK"] = eq_w <= equity_max + 1e-8

    return pd.Series(out)

incremental_constraint_report(
    w_opt=w_inc,
    w_current=w_current,
    max_asset_weight=0.40,
    max_active_weight=0.10,
    illiquid_tickers=("FECRX", "FERLX"),
    equity_tickers=("FEGIX", "SGOIX", "FEAIX", "FESCX", "FESMX"),
)


Sum of weights             1.0
Max single-name weight    0.24
Min single-name weight     0.0
Max |active weight|        0.1
SGOV weight               0.02
SGOV within band          True
FEHIX cap OK              True
HYG cap OK                True
FEGIX cap OK              True
Illiquid bucket weight     0.0
Illiquid cap OK           True
Equity bucket weight      0.52
Equity cap OK             True
dtype: object

## Optimized Portfolio (Risk Budget (Max RC Share=30%) Approach)

In [256]:
import numpy as np
import pandas as pd

def risk_contribution_share(
    weights: pd.Series,
    cov_annual: pd.DataFrame,
) -> pd.Series:
    """
    Compute total risk contribution share per asset under annual covariance.
    RC_i = w_i * (Sigma w)_i
    Share_i = RC_i / (w' Sigma w)
    """
    tickers = list(cov_annual.columns)
    w = weights.reindex(tickers).fillna(0.0).astype(float).values
    Sigma = cov_annual.loc[tickers, tickers].astype(float).values

    # Portfolio variance
    port_var = float(w @ Sigma @ w)
    if port_var <= 0:
        return pd.Series(0.0, index=tickers, name="Risk_Contribution_Share")

    # Marginal contribution to variance: (Sigma w)
    mrc = Sigma @ w

    # Total contribution to variance: w_i * (Sigma w)_i
    rc = w * mrc

    # Normalize to shares
    share = rc / port_var
    return pd.Series(share, index=tickers, name="Risk_Contribution_Share")


In [257]:
from scipy.optimize import minimize

def optimize_risk_budget_portfolio(
    mu_annual: pd.Series,
    cov_annual: pd.DataFrame,
    w_current: pd.Series,
    risk_aversion: float = 3.0,
    max_risk_share: float = 0.30,      # Risk Contribution Share cap
    max_asset_weight: float = 0.40,    # Single-name cap
    sgov_min: float = 0.02,
    sgov_max: float = 0.10,
    fehix_max: float = 0.20,
    hyg_max: float = 0.20,
    fegix_max: float = 0.20,
    illiquid_max: float = 0.20,
    equity_max: float = 0.60,
    illiquid_tickers: tuple = ("FECRX", "FERLX"),
    equity_tickers: tuple = ("FEGIX", "SGOIX", "FEAIX", "FESCX", "FESMX"),
    verbose: bool = True,
) -> dict:
    """
    Risk Budget approach:
    - Remove active-weight (Δw) constraint
    - Add per-asset risk contribution share constraint: RC_share_i <= max_risk_share
    - Keep long-only, single-name caps, and policy bucket caps
    - Maximize mean-variance utility: mu'w - 0.5*risk_aversion*(w'Σw)
    """

    # Universe alignment
    tickers = list(cov_annual.columns)
    cov = cov_annual.loc[tickers, tickers].astype(float)
    Sigma = cov.values

    mu = mu_annual.reindex(tickers).astype(float)
    if mu.isna().any():
        missing = mu[mu.isna()].index.tolist()
        raise ValueError(f"Missing expected returns for tickers: {missing}")
    mu_v = mu.values

    # Initialization (use current portfolio as a stable starting point)
    w0 = w_current.reindex(tickers).fillna(0.0).astype(float)
    w0 = w0 / w0.sum()
    x0 = np.clip(w0.values.copy(), 0.0, max_asset_weight)
    x0 = x0 / x0.sum()

    # Objective: maximize utility => minimize negative utility
    def neg_utility(w: np.ndarray) -> float:
        w = np.asarray(w, dtype=float)
        port_ret = float(w @ mu_v)
        port_var = float(w @ Sigma @ w)
        utility = port_ret - 0.5*risk_aversion * port_var
        return -utility

    constraints = []

    # Fully invested
    constraints.append({"type": "eq", "fun": lambda w: np.sum(w) - 1.0})

    # SGOV allocation band
    if "SGOV" in tickers:
        idx = tickers.index("SGOV")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: w[idx] - sgov_min})
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: sgov_max - w[idx]})

    # Single-fund caps (policy constraints)
    if "FEHIX" in tickers:
        idx = tickers.index("FEHIX")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: fehix_max - w[idx]})

    if "HYG" in tickers:
        idx = tickers.index("HYG")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: hyg_max - w[idx]})

    if "FEGIX" in tickers:
        idx = tickers.index("FEGIX")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: fegix_max - w[idx]})

    # Bucket caps (illiquid / equity)
    ill_idx = [tickers.index(t) for t in illiquid_tickers if t in tickers]
    if ill_idx:
        constraints.append({"type": "ineq", "fun": lambda w, ii=ill_idx: illiquid_max - np.sum(w[ii])})

    eq_idx = [tickers.index(t) for t in equity_tickers if t in tickers]
    if eq_idx:
        constraints.append({"type": "ineq", "fun": lambda w, ei=eq_idx: equity_max - np.sum(w[ei])})

    # Risk contribution share constraint: for each asset i, max_risk_share - share_i(w) >= 0
    # share_i(w) = [w_i * (Sigma w)_i] / (w' Sigma w)
    def make_rc_share_ineq(i: int):
        def _ineq(w: np.ndarray) -> float:
            w = np.asarray(w, dtype=float)
            port_var = float(w @ Sigma @ w)
            if port_var <= 0:
                return max_risk_share  # trivially feasible if variance is ~0
            mrc = Sigma @ w
            share_i = (w[i] * mrc[i]) / port_var
            return max_risk_share - share_i
        return _ineq

    for i in range(len(tickers)):
        constraints.append({"type": "ineq", "fun": make_rc_share_ineq(i)})

    # Bounds: long-only and single-name cap
    bounds = [(0.0, max_asset_weight) for _ in tickers]

    res = minimize(
        fun=neg_utility,
        x0=x0,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": 4000, "ftol": 1e-12, "disp": verbose},
    )

    if not res.success:
        raise RuntimeError(f"Risk Budget optimization failed: {res.message}")

    w_opt = pd.Series(res.x, index=tickers, name="Weight")

    return {
        "weights": w_opt,
        "objective_value": float(-res.fun),
        "solver_result": res,
    }


## Theoratical Approach (no constraints on Δw and risk contribution share)

In [265]:
def optimize_theoretical_optimal_portfolio(
    mu_annual: pd.Series,
    cov_annual: pd.DataFrame,
    w_current: pd.Series,
    risk_aversion: float = 3.0,
    max_asset_weight: float = 0.40,
    sgov_min: float = 0.02,
    sgov_max: float = 0.10,
    fehix_max: float = 0.20,
    hyg_max: float = 0.20,
    fegix_max: float = 0.20,
    illiquid_max: float = 0.20,
    equity_max: float = 0.60,
    illiquid_tickers: tuple = ("FECRX", "FERLX"),
    equity_tickers: tuple = ("FEGIX", "SGOIX", "FEAIX", "FESCX", "FESMX"),
    verbose: bool = True,
) -> dict:
    """
    Theoretical optimal portfolio:
    - Remove active-weight (Δw) constraint
    - Do NOT impose risk contribution share constraint
    - Keep long-only, single-name caps, and policy bucket caps
    - Maximize mean-variance utility: mu'w - 0.5* risk_aversion*(w'Σw)
    """

    tickers = list(cov_annual.columns)
    Sigma = cov_annual.loc[tickers, tickers].astype(float).values

    mu = mu_annual.reindex(tickers).astype(float)
    if mu.isna().any():
        missing = mu[mu.isna()].index.tolist()
        raise ValueError(f"Missing expected returns for tickers: {missing}")
    mu_v = mu.values

    w0 = w_current.reindex(tickers).fillna(0.0).astype(float)
    w0 = w0 / w0.sum()
    x0 = np.clip(w0.values.copy(), 0.0, max_asset_weight)
    x0 = x0 / x0.sum()

    def neg_utility(w: np.ndarray) -> float:
        w = np.asarray(w, dtype=float)
        port_ret = float(w @ mu_v)
        port_var = float(w @ Sigma @ w)
        utility = port_ret - 0.5*risk_aversion * port_var
        return -utility

    constraints = []
    constraints.append({"type": "eq", "fun": lambda w: np.sum(w) - 1.0})

    if "SGOV" in tickers:
        idx = tickers.index("SGOV")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: w[idx] - sgov_min})
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: sgov_max - w[idx]})

    if "FEHIX" in tickers:
        idx = tickers.index("FEHIX")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: fehix_max - w[idx]})

    if "HYG" in tickers:
        idx = tickers.index("HYG")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: hyg_max - w[idx]})

    if "FEGIX" in tickers:
        idx = tickers.index("FEGIX")
        constraints.append({"type": "ineq", "fun": lambda w, idx=idx: fegix_max - w[idx]})

    ill_idx = [tickers.index(t) for t in illiquid_tickers if t in tickers]
    if ill_idx:
        constraints.append({"type": "ineq", "fun": lambda w, ii=ill_idx: illiquid_max - np.sum(w[ii])})

    eq_idx = [tickers.index(t) for t in equity_tickers if t in tickers]
    if eq_idx:
        constraints.append({"type": "ineq", "fun": lambda w, ei=eq_idx: equity_max - np.sum(w[ei])})

    bounds = [(0.0, max_asset_weight) for _ in tickers]

    res = minimize(
        fun=neg_utility,
        x0=x0,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": 4000, "ftol": 1e-12, "disp": verbose},
    )

    if not res.success:
        raise RuntimeError(f"Theoretical optimization failed: {res.message}")

    w_opt = pd.Series(res.x, index=tickers, name="Weight")

    return {
        "weights": w_opt,
        "objective_value": float(-res.fun),
        "solver_result": res,
    }


In [266]:
def risk_contribution_summary(weights: pd.Series, cov_annual: pd.DataFrame) -> dict:
    """
    Produce risk concentration diagnostics:
    - Max risk contribution share
    - Top-3 risk contribution share sum
    - Effective Number of Bets (ENB) = 1 / sum(share_i^2)
    """
    share = risk_contribution_share(weights, cov_annual).clip(lower=0.0)
    max_share = float(share.max())
    top3_sum = float(share.sort_values(ascending=False).head(3).sum())
    hhi = float((share ** 2).sum())
    enb = (1.0 / hhi) if hhi > 0 else np.nan

    return {
        "Max Risk Contribution Share": max_share,
        "Top 3 Risk Share Sum": top3_sum,
        "Effective Number of Bets (ENB)": enb,
    }


## Portfolio Metrics Summary Table

In [277]:
import numpy as np
import pandas as pd

# ============================================================
# 0) USER-DEFINED DISPLAY ORDER (Investment Universe)
# ============================================================
ASSET_ORDER = [
    "FECRX",
    "SGIIX",
    "FEBIX",
    "FEREX",
    "FEGIX",
    "FEHIX",
    "SGOIX",
    "FERLX",
    "FEAIX",
    "FDUIX",
    "FESCX",
    "FESMX",
    "FEVIX",
    "HYG",
    "LQD",
    "SGOV",
]

# ============================================================
# 1) CORE METRICS FUNCTIONS
#    (Assumes annualized mu and annualized covariance matrix)
# ============================================================
def portfolio_stats(
    weights: pd.Series,
    mu: pd.Series,
    cov: pd.DataFrame,
    rf: float = 0.0
) -> dict:
    """
    Compute portfolio expected return, variance, volatility, and Sharpe ratio.
    All inputs are assumed annualized (mu and cov).
    """
    tickers = list(cov.columns)
    w = weights.reindex(tickers).fillna(0.0).astype(float).values
    mu_v = mu.reindex(tickers).astype(float).values
    Sigma = cov.loc[tickers, tickers].astype(float).values

    port_ret = float(w @ mu_v)
    port_var = float(w @ Sigma @ w)
    port_vol = float(np.sqrt(max(port_var, 0.0)))
    sharpe = (port_ret - rf) / port_vol if port_vol > 0 else np.nan

    return {
        "return": port_ret,
        "variance": port_var,
        "vol": port_vol,
        "sharpe": sharpe
    }


def risk_contribution_table(
    weights: pd.Series,
    cov: pd.DataFrame
) -> pd.DataFrame:
    """
    Compute per-asset marginal risk contribution (MRC),
    total risk contribution (TRC), and risk contribution share (RC Share).

    Definitions:
      - Portfolio volatility: sigma_p = sqrt(w' Σ w)
      - Marginal contribution to risk: MRC_i = (Σ w)_i / sigma_p
      - Total contribution to risk: TRC_i = w_i * MRC_i
      - RC Share_i = TRC_i / sigma_p
    """
    tickers = list(cov.columns)
    w = weights.reindex(tickers).fillna(0.0).astype(float)
    Sigma = cov.loc[tickers, tickers].astype(float)

    port_var = float(w.values @ Sigma.values @ w.values)
    port_vol = float(np.sqrt(max(port_var, 0.0)))

    # Handle degenerate case
    if port_vol <= 0:
        out = pd.DataFrame(index=tickers)
        out["Weight"] = w.values
        out["MRC"] = np.nan
        out["TRC"] = np.nan
        out["RC Share"] = np.nan
        return out

    # Σw
    Sigma_w = Sigma.values @ w.values  # shape (n,)

    # MRC_i = (Σw)_i / sigma_p
    mrc = Sigma_w / port_vol

    # TRC_i = w_i * MRC_i
    trc = w.values * mrc

    # RC Share_i = TRC_i / sigma_p
    rc_share = trc / port_vol

    out = pd.DataFrame(index=tickers)
    out["Weight"] = w.values
    out["RC Share"] = rc_share
    return out


def risk_contribution_summary(weights: pd.Series, cov: pd.DataFrame) -> dict:
    """
    Summarize risk concentration:
      - Max RC Share
      - Top 3 RC Share sum
      - Effective Number of Bets (ENB) = 1 / sum(RC_share_i^2)
    """
    rc = risk_contribution_table(weights, cov)
    rc_share = rc["RC Share"].dropna()

    if rc_share.empty:
        return {
            "Max Risk Contribution Share": np.nan,
            "Top 3 Risk Share Sum": np.nan,
            "Effective Number of Bets (ENB)": np.nan
        }

    rc_sorted = rc_share.sort_values(ascending=False)
    max_rc = float(rc_sorted.iloc[0])
    top3 = float(rc_sorted.iloc[:3].sum())
    enb = float(1.0 / np.sum(rc_share.values ** 2)) if np.sum(rc_share.values ** 2) > 0 else np.nan

    return {
        "Max Risk Contribution Share": max_rc,
        "Top 3 Risk Share Sum": top3,
        "Effective Number of Bets (ENB)": enb
    }


def portfolio_metrics_row(
    name: str,
    weights: pd.Series,
    mu_annual: pd.Series,
    cov_annual: pd.DataFrame,
    rf: float = 0.0
) -> pd.Series:
    """
    Consolidate performance + risk concentration metrics into a single row.
    """
    stats = portfolio_stats(weights, mu_annual, cov_annual, rf=rf)
    rc_sum = risk_contribution_summary(weights, cov_annual)

    return pd.Series({
        "Expected Return (Ann.)": stats["return"],
        "Volatility (Ann.)": stats["vol"],
        "Variance (Ann.)": stats["variance"],
        "Sharpe (Rf=0)": stats["sharpe"],
        "Max RC Share": rc_sum["Max Risk Contribution Share"],
        "Top 3 RC Share": rc_sum["Top 3 Risk Share Sum"],
        "ENB (1/HHI)": rc_sum["Effective Number of Bets (ENB)"],
    }, name=name)


# ============================================================
# 2) INPUT: MAKE SURE THESE EXIST IN YOUR NOTEBOOK
# ============================================================
# Required:
#   mu_annual: pd.Series
#   cov_annual: pd.DataFrame
#   w_current: pd.Series
#   w_inc: pd.Series
#   w_rb: pd.Series
#   w_th: pd.Series
#
# If your variables are named differently, map them here.

PORTFOLIOS = {
    "Current": w_current,
    "Incremental (Δw-Constrained)": w_inc,
    "Risk Budget (Max RC Share=30%)": w_rb,
    "Theoretical Optimal": w_th,
}


# ============================================================
# 3) SUMMARY TABLE (PORTFOLIO-LEVEL)
# ============================================================
rows = []
for name, w in PORTFOLIOS.items():
    rows.append(portfolio_metrics_row(name, w, mu_annual, cov_annual, rf=0.0))

summary_all = pd.DataFrame(rows)

# Finance-friendly formatting for presentation
summary_fmt = summary_all.copy()
pct_cols = ["Expected Return (Ann.)", "Volatility (Ann.)", "Max RC Share", "Top 3 RC Share"]
for c in pct_cols:
    summary_fmt[c] = summary_fmt[c].map(lambda x: f"{x:.2%}")
summary_fmt["Sharpe (Rf=0)"] = summary_fmt["Sharpe (Rf=0)"].map(lambda x: f"{x:.2f}")
summary_fmt["Variance (Ann.)"] = summary_fmt["Variance (Ann.)"].map(lambda x: f"{x:.4f}")
summary_fmt["ENB (1/HHI)"] = summary_fmt["ENB (1/HHI)"].map(lambda x: f"{x:.2f}")

print("=== Portfolio-Level Summary ===")
display(summary_fmt)


# ============================================================
# 4) ASSET-LEVEL COMPARISON TABLE
#    - Weight, Δw vs Current, and RC Share
#    - Strictly ordered by ASSET_ORDER
# ============================================================
# Build a union index from covariance columns (true universe) to keep math consistent
universe = list(cov_annual.columns)

# Safety: keep only assets actually in universe
asset_order_valid = [t for t in ASSET_ORDER if t in universe]

# Pre-compute current (for Δw baseline)
w_cur_aligned = w_current.reindex(universe).fillna(0.0).astype(float)

# Create the asset-level table
asset_table = pd.DataFrame(index=universe)

for name, w in PORTFOLIOS.items():
    w_aligned = w.reindex(universe).fillna(0.0).astype(float)

    # Risk contribution shares for this portfolio
    rc_tbl = risk_contribution_table(w_aligned, cov_annual)
    rc_share = rc_tbl["RC Share"].reindex(universe)

    # Columns
    asset_table[(name, "Weight")] = w_aligned.values
    asset_table[(name, "Δw vs Current")] = (w_aligned - w_cur_aligned).values
    asset_table[(name, "RC Share")] = rc_share.values

# Make it a clean MultiIndex column table
asset_table.columns = pd.MultiIndex.from_tuples(asset_table.columns, names=["Portfolio", "Metric"])

# Reorder rows to match your required investment universe order
asset_table = asset_table.reindex(asset_order_valid)

# Format for presentation
asset_table_fmt = asset_table.copy()

for p in PORTFOLIOS.keys():
    asset_table_fmt[(p, "Weight")] = asset_table_fmt[(p, "Weight")].map(lambda x: f"{x:.0%}")
    asset_table_fmt[(p, "Δw vs Current")] = asset_table_fmt[(p, "Δw vs Current")].map(lambda x: f"{x:+.0%}")
    asset_table_fmt[(p, "RC Share")] = asset_table_fmt[(p, "RC Share")].map(lambda x: f"{x:.0%}" if pd.notna(x) else "")

print("=== Asset-Level Comparison ===")
display(asset_table_fmt)


=== Portfolio-Level Summary ===


,Expected Return (Ann.),Volatility (Ann.),Variance (Ann.),Sharpe (Rf=0),Max RC Share,Top 3 RC Share,ENB (1/HHI)
Current,21.56%,10.82%,0.0117,1.99,41.14%,80.45%,3.59
Incremental (Δw-Constrained),35.36%,13.33%,0.0178,2.65,38.45%,74.46%,4.17
Risk Budget (Max RC Share=30%),49.19%,16.40%,0.0269,3.00,30.00%,88.08%,3.71
Theoretical Optimal,49.25%,16.38%,0.0268,3.01,31.73%,89.48%,3.59


=== Asset-Level Comparison ===


Portfolio Current                        Incremental (Δw-Constrained)  \
Metric     Weight Δw vs Current RC Share                       Weight   
FECRX          0%           +0%       0%                           0%   
SGIIX          9%           +0%      12%                           6%   
FEBIX          0%           +0%       0%                           0%   
FEREX          0%           +0%       0%                           0%   
FEGIX          0%           +0%       0%                          10%   
FEHIX          0%           +0%      -0%                           0%   
SGOIX          8%           +0%       8%                          18%   
FERLX          0%           +0%      -0%                           0%   
FEAIX          0%           +0%       0%                           0%   
FDUIX          7%           +0%      -0%                           0%   
FESCX         14%           +0%      28%                          24%   
FESMX          0%           +0%       0%                           0%   
FEVIX         29%           +0%      41%                          19%   
HYG            0%           +0%       0%                           0%   
LQD           31%           +0%      12%                          21%   
SGOV           2%           +0%      -0%                           2%   

Portfolio                        Risk Budget (Max RC Share=30%)                \
Metric    Δw vs Current RC Share                         Weight Δw vs Current   
FECRX               +0%       0%                             0%           +0%   
SGIIX               -3%       6%                            36%          +27%   
FEBIX               +0%       0%                             0%           +0%   
FEREX               +0%       0%                             2%           +2%   
FEGIX              +10%      14%                            20%          +20%   
FEHIX               +0%      -0%                             0%           +0%   
SGOIX              +10%      15%                            14%           +6%   
FERLX               +0%       0%                             0%           +0%   
FEAIX               +0%       0%                             0%           +0%   
FDUIX               -7%      -0%                             0%           -7%   
FESCX              +10%      38%                            26%          +12%   
FESMX               +0%       0%                             0%           +0%   
FEVIX              -10%      21%                             0%          -29%   
HYG                 +0%       0%                             0%           +0%   
LQD                -10%       6%                             0%          -31%   
SGOV                +0%      -0%                             2%           -0%   

Portfolio          Theoretical Optimal                         
Metric    RC Share              Weight Δw vs Current RC Share  
FECRX           0%                  0%           +0%       0%  
SGIIX          30%                 38%          +29%      32%  
FEBIX           0%                  0%           +0%       0%  
FEREX           2%                  0%           +0%       0%  
FEGIX          28%                 20%          +20%      28%  
FEHIX          -0%                  0%           +0%      -0%  
SGOIX          10%                 15%           +7%      11%  
FERLX           0%                  0%           +0%       0%  
FEAIX           0%                  0%           +0%       0%  
FDUIX          -0%                  0%           -7%      -0%  
FESCX          30%                 25%          +11%      30%  
FESMX           0%                  0%           +0%       0%  
FEVIX           0%                  0%          -29%       0%  
HYG             0%                  0%           +0%       0%  
LQD             0%                  0%          -31%       0%  
SGOV           -0%                  2%           +0%      -0%